## Programming Assignment - Retrieving Wikipedia Articles

In [1]:
import turicreate as tc

In [3]:
articles = tc.SFrame('people_wiki.sframe')
articles.head()

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


In [25]:
articles['word_count'] = tc.text_analytics.count_words(articles['text'])
articles['tf_idf'] = tc.text_analytics.tf_idf(articles['text'])

### 1. Compare top words according to word counts to TF-IDF

Select only articles about Elton John. What are the 3 words in his articles with highest word counts?  What are the 3 words in his articles with highest TF-IDF?

In [26]:
elton = articles[articles['name'] == 'Elton John']
elton

Columns:
	URI	str
	name	str
	text	str
	tf_idf	dict
	word_count	dict

Rows: Unknown

Data:
+-------------------------------+------------+-------------------------------+
|              URI              |    name    |              text             |
+-------------------------------+------------+-------------------------------+
| <http://dbpedia.org/resour... | Elton John | sir elton hercules john cb... |
+-------------------------------+------------+-------------------------------+
+-------------------------------+-------------------------------+
|             tf_idf            |           word_count          |
+-------------------------------+-------------------------------+
| {'movements': 5.0306580197... | {'movements': 1.0, 'social... |
+-------------------------------+-------------------------------+
[? rows x 5 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [27]:
elton['text']

dtype: str
Rows: ?
['sir elton hercules john cbe born reginald kenneth dwight 25 march 1947 is an english singer songwriter composer pianist record producer and occasional actor he has worked with lyricist bernie taupin as his songwriter partner since 1967 they have collaborated on more than 30 albums to datein his fivedecade career elton john has sold more than 300 million records making him one of the bestselling music artists in the world he has more than fifty top 40 hits including seven consecutive no 1 us albums 58 billboard top 40 singles 27 top 10 four no 2 and nine no 1 for 31 consecutive years 19702000 he had at least one song in the billboard hot 100 his single something about the way you look tonightcandle in the wind 1997 sold over 33 million copies worldwide and is the bestselling single of all time he has received six grammy awards five brit awards winning two awards for outstanding contribution to music and the first brits icon in 2013 for his lasting impact on british 

In [28]:
word_count_table = elton[['word_count']].stack('word_count', new_column_name = ['word', 'count'])
word_count_table.sort('count', ascending = False)

word,count
the,27.0
in,18.0
and,15.0
of,13.0
a,10.0
has,9.0
he,7.0
john,7.0
on,6.0
award,5.0


The top three words with the highest counts are 'the', 'in' and 'and'.

In [29]:
tf_idf_table = elton[['tf_idf']].stack('tf_idf', new_column_name = ['word', 'tfidf'])
tf_idf_table.sort('tfidf', ascending = False)

word,tfidf
furnish,18.38947183999428
elton,17.482320270031995
billboard,17.30368095754203
john,13.93931279239831
songwriters,11.250406447031539
overallelton,10.986495389225194
tonightcandle,10.986495389225194
fivedecade,10.293348208665249
19702000,10.293348208665249
aids,10.262846934045534


The top three words with the highest tf-idf are 'five', 'worldwide' and 'charitable'.

(or if you calculate tf_idf on all articles and then subset out Elton, it's 'furnish', 'elton' and 'billboard')

### 2. Measuring distance

Compare the cosine distance between the article about Elton John and the articles about Victoria Beckham and Paul McCartney. Which one of the two is closest? Does this make sense?

In [22]:
articles['tf_idf'] = tc.text_analytics.tf_idf(articles['text'])
articles

URI,name,text,tf_idf
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'melbourne':3.8914310119380633, ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'time':1.3253342074200498, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'society':2.4448047262085693, ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'kurdlawitzpreis':10.986495389225194, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'curtis':5.299520032885375, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'asses':9.600201028105303, 's ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'streamz':10.986495389225194, ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'concordia':6.250296940830698, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'heavies':8.907053847545358, 'n ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'2002':1.8753125887822302, ..."


In [23]:
victoria = articles[articles['name'] == 'Victoria Beckham']
paul = articles[articles['name'] == 'Paul McCartney']
elton = articles[articles['name'] == 'Elton John']

In [24]:
print(tc.distances.cosine(elton['tf_idf'][0], victoria['tf_idf'][0]))
print(tc.distances.cosine(elton['tf_idf'][0], paul['tf_idf'][0]))

0.9567006376655429
0.8250310029221779


This appears to be slightly strange, except that Victoria Beckham was a member of the Spice Girls... so maybe not.

### 3.     Building nearest neighbors models with different input features and setting the distance metric

Build two nearest neighbours models: one using word counts as features and the other using TF-IDF as features. In both models, set the distance function to cosine similarity.

Use the models to answer the following questions:

- What’s the most similar article, other than itself, to the one on ‘Elton John’ using word count features?
- What’s the most similar article, other than itself, to the one on ‘Elton John’ using TF-IDF features?
- What’s the most similar article, other than itself, to the one on ‘Victoria Beckham’ using word count features?
- What’s the most similar article, other than itself, to the one on ‘Victoria Beckham’ using TF-IDF features?

In [30]:
articles['word_count'] = tc.text_analytics.count_words(articles['text'])

In [31]:
knn_tfidf_model = tc.nearest_neighbors.create(articles, features = ['tf_idf'], label = 'name', distance = 'cosine')
knn_wc_model = tc.nearest_neighbors.create(articles, features = ['word_count'], label = 'name', distance = 'cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [32]:
knn_tfidf_model

Class                          : NearestNeighborsModel

Attributes
----------
Method                         : brute_force
Number of distance components  : 1
Number of examples             : 59071
Number of feature columns      : 1
Number of unpacked features    : 547979
Distance                       : cosine
Total training time (seconds)  : 15.4511

In [33]:
knn_wc_model

Class                          : NearestNeighborsModel

Attributes
----------
Method                         : brute_force
Number of distance components  : 1
Number of examples             : 59071
Number of feature columns      : 1
Number of unpacked features    : 547979
Distance                       : cosine
Total training time (seconds)  : 14.266

In [34]:
elton = articles[articles['name'] == 'Elton John']
knn_tfidf_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.316ms      |

| Done         |         | 100         | 467.135ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.220446049250313e-16,1
0,Rod Stewart,0.7172196678927374,2
0,George Michael,0.7476009989692848,3
0,Sting (musician),0.7476719544306141,4
0,Phil Collins,0.7511932487904706,5


In [35]:
knn_wc_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 2.738ms      |

| Done         |         | 100         | 379.005ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.220446049250313e-16,1
0,Cliff Richard,0.16142415258967036,2
0,Sandro Petrone,0.16822542751041114,3
0,Rod Stewart,0.16832716558706107,4
0,Malachi O'Doherty,0.177315545978884,5


- the most similar article to Elton John's using word counts as features is Cliff Richard
- the most similar article to Elton John's using tf-idf as features is Rod Stewart

In [40]:
victoria = articles[articles['name'] == 'Victoria Beckham']
knn_tfidf_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 13.397ms     |

| Done         |         | 100         | 421.297ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.1102230246251565e-16,1
0,David Beckham,0.5481696102632145,2
0,Stephen Dow Beckham,0.7849867068283364,3
0,Mel B,0.8095855234085036,4
0,Caroline Rush,0.81982642291868,5


In [41]:
knn_wc_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 3.625ms      |

| Done         |         | 100         | 384.675ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.220446049250313e-16,1
0,Mary Fitzgerald (artist),0.20730703611504997,2
0,Adrienne Corri,0.21450978278754795,3
0,Beverly Jane Fry,0.21746646874079278,4
0,Raman Mundair,0.21769547499150488,5


- the most similar article to Victoria Beckham's using word counts as features is Mary Fitzgerald
- the most similar article to Elton John's using tf-idf as features is David Beckham